# Financial Analysis for ETFs and Mutual Funds

## ETFs (Exchange-Traded Funds) vs Mutual Funds 

Both popular investment vehicles that pool money from multiple investors to invest in a diversified portfolio of assets. 
 
Key differences:

1. Trading:
   - ETFs: Traded on stock exchanges throughout the day at market prices.
   - Mutual Funds: Priced and traded once per day after market close.

2. Pricing:
   - ETFs: Prices fluctuate throughout the trading day.
   - Mutual Funds: Priced at the Net Asset Value (NAV) calculated at the end of each trading day.

3. Minimum Investment:
   - ETFs: Can be purchased for the price of a single share.
   - Mutual Funds: Often have minimum initial investment requirements.

4. Management:
   - ETFs: Mostly passively managed, tracking an index.
   - Mutual Funds: Can be actively or passively managed.

5. Expense Ratios:
   - ETFs: Generally have lower expense ratios.
   - Mutual Funds: Often have higher expense ratios, especially for actively managed funds.

6. Tax Efficiency:
   - ETFs: Usually more tax-efficient due to their structure and lower turnover.
   - Mutual Funds: May be less tax-efficient, particularly actively managed funds.

7. Transparency:
   - ETFs: Holdings are typically disclosed daily.
   - Mutual Funds: Holdings are usually disclosed quarterly or semi-annually.


## Types of Fees

1. Expense Ratio:
   - This is the annual fee that all funds charge their shareholders.
   - It expresses the percentage of assets deducted each fiscal year for fund expenses.
   - Includes management fees, administrative fees, and most operating costs.
   - Generally, ETFs have lower expense ratios than mutual funds.

2. Management Fees:
   - Compensation for the fund manager, included in the expense ratio.
   - Generally higher for actively managed funds.

3. Sales Loads (more common in mutual funds):
   - Front-end load: A fee paid when you buy shares.
   - Back-end load: A fee paid when you sell shares.
   - Many funds now offer no-load options.

4. Transaction Fees:
   - For ETFs: Brokerage commissions when buying or selling shares.
   - For Mutual Funds: Some brokers charge a fee to buy or sell certain mutual funds.

5. Trading Costs:
   - Costs incurred by the fund when buying and selling securities.
   - Not included in the expense ratio but affects returns.


## Expense Ratio Charge
- The expense ratio is charged daily, not annually or monthly.
- It's calculated as a percentage of the fund's average net assets for the year, divided by 365 (or 366 in a leap year).
- It's not visible on the account statement. Instead, it reduces the fund's overall return, which in turn affects the value of your investment.

## Simulation

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests

In [2]:
ticker ='VPC'
api_key = 'M66ZYA8XASEVDYE6'
url = 'https://www.alphavantage.co/query?function=ETF_PROFILE&symbol=QQQ&apikey=demo'
url = url.replace('QQQ', ticker).replace('demo',api_key)
r = requests.get(url)
data = r.json()

print(data['net_expense_ratio'])

0.0972


$${Net Return} = {Annual Return} - (Expense Ratio + Management Fee)$$
$${Investment Value} = {Initial Investment} * (1 + {Net Return})^{Number of Periods}$$

In [3]:


# Function to simulate investment growth with expense ratio and management fee
def simulate_investment(initial_investment, years, annual_return, expense_ratio, management_fee):
    # Create a DataFrame to store investment values for each year
    data = {'Year': np.arange(0, years + 1)}
    df = pd.DataFrame(data)
    
    # Calculate net return 
    net_return = annual_return - (expense_ratio + management_fee)
    
    # Calculate the investment value over time
    df['Investment Value'] = initial_investment * (1 + net_return) ** df['Year']
    
    return df


In [4]:

# Adjustable parameters
initial_investment = 10000  
years = 20  # Investment period
annual_return = 0.07  # % annual return before fees
baseline_expense_ratio = 0.01  # Fixed baseline expense ratio at 1%
management_fee = 0.01  # Fixed management fee (1%)

# Simulate investment for baseline expense ratio
baseline_df = simulate_investment(initial_investment, years, annual_return, baseline_expense_ratio, management_fee)
baseline_df.head()

,Year,Investment Value
0,0,10000.0000
1,1,10500.0000
2,2,11025.0000
3,3,11576.2500
4,4,12155.0625


In [5]:

# Expense ratios to include in the dropdown
expense_ratios = [0.005, 0.01, 0.015, 0.02, 0.025]  # Different expense ratios (0.5% to 2.5%)

# Create a figure
fig = go.Figure()

# Add the baseline line (fixed expense ratio)
fig.add_trace(go.Scatter(
    x=baseline_df['Year'],
    y=baseline_df['Investment Value'],
    mode='lines',
    name=f'Baseline (1%)',
    line=dict(color='royalblue')
))

# Add traces for each expense ratio for comparison (only one will be visible at a time)
for ratio in expense_ratios:
    df = simulate_investment(initial_investment, years, annual_return, ratio, management_fee)
    fig.add_trace(go.Scatter(
        x=df['Year'],
        y=df['Investment Value'],
        mode='lines',
        name=f'Expense Ratio: {ratio*100:.1f}%',
        visible=False  # Initially, only the baseline is visible
    ))

# Create drop-down menu
dropdown_buttons = [
    dict(
        args=[{'visible': [True] + [i == idx for i in range(len(expense_ratios))]}],  # Toggle visibility
        label=f'Exp. Ratio: {expense_ratios[idx]*100}%',
        method='update'
    ) for idx in range(len(expense_ratios))
]

# Update layout with drop-down
fig.update_layout(
    updatemenus=[{
        'active': 0,  # Baseline is initially active
        'buttons': dropdown_buttons,
        'xanchor':'left',
        'x': 0.01, 'y': 1.15  # Position of the drop-down menu
        
    }],
    legend=dict(
        orientation="h",  # Make legend horizontal
        yanchor="bottom",  # Anchor the legend to the top
        y=1.02,  # Position it above the plot
        xanchor="right",  # Align legend to the right
        x=1,  # Place the legend to the right of the title
    ),
    title="Impact of Expense Ratio on a $10,000 Investment",
    xaxis_title="Years",
    yaxis_title="Investment Value (USD)",
    template="plotly_dark"
)

# Show plot
fig.show()


## With Annual Return Rate Simulation Added

In [8]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Function to simulate investment growth with expense ratio and management fee
def simulate_investment(initial_investment, years, annual_return, expense_ratio, management_fee):
    # Create a DataFrame to store investment values for each year
    data = {'Year': np.arange(0, years + 1)}
    df = pd.DataFrame(data)
    
    # Calculate net return 
    net_return = annual_return - (expense_ratio + management_fee)
    
    # Calculate the investment value over time
    df['Investment Value'] = initial_investment * (1 + net_return) ** df['Year']
    
    return df

# Adjustable parameters
initial_investment = 10000  
years = 20  # Investment period
baseline_annual_return = 0.07  # Baseline annual return (7%)
baseline_expense_ratio = 0.01  # Baseline expense ratio (1%)
management_fee = 0.01  # Fixed management fee (1%)

# Simulate investment for baseline
baseline_df = simulate_investment(initial_investment, years, baseline_annual_return, baseline_expense_ratio, management_fee)

# Expense ratios and annual returns to include in the dropdowns
expense_ratios = [0.005, 0.01, 0.015, 0.02, 0.025]  # Different expense ratios (0.5% to 2.5%)
annual_returns = [0.06, 0.07, 0.08, 0.09, 0.10, 0.15]  # Different annual returns (5% to 9%)

# Create a figure
fig = go.Figure()

# Add the baseline line (7% return, 1% expense ratio)
fig.add_trace(go.Scatter(
    x=baseline_df['Year'],
    y=baseline_df['Investment Value'],
    mode='lines',
    name=f'Baseline (7% return, 1% exp. ratio)',
    line=dict(color='royalblue')
))

# Add traces for each expense ratio and return rate (hidden by default)
for ratio in expense_ratios:
    for ret in annual_returns:
        df = simulate_investment(initial_investment, years, ret, ratio, management_fee)
        fig.add_trace(go.Scatter(
            x=df['Year'],
            y=df['Investment Value'],
            mode='lines',
            name=f'Exp. Ratio: {ratio*100:.1f}%, Return: {ret*100:.1f}%',
            visible=False  # Initially hidden
        ))

# Create drop-down menu for expense ratios
expense_ratio_dropdown = [
    dict(
        args=[{'visible': [True] + [i == idx for i in range(len(expense_ratios) * len(annual_returns))]}],  # Toggle visibility
        label=f'Exp. Ratio: {expense_ratios[idx]*100}%',
        method='update'
    ) for idx in range(len(expense_ratios))
]

# Create drop-down menu for annual returns
annual_return_dropdown = [
    dict(
        args=[{'visible': [True] + [i % len(annual_returns) == idx for i in range(len(expense_ratios) * len(annual_returns))]}],
        label=f'Annual Return: {annual_returns[idx]*100:.1f}%',
        method='update'
    ) for idx in range(len(annual_returns))
]

# Update layout with two drop-down menus
fig.update_layout(
    updatemenus=[
        {
            'buttons': expense_ratio_dropdown,
            'xanchor': 'left',
            'x': 0.01, 'y': 1.15,  # Position of the expense ratio drop-down
            'direction': 'down',
            'showactive': True
        },
        {
            'buttons': annual_return_dropdown,
            'xanchor': 'left',
            'x': 0.2, 'y': 1.15,  # Position of the annual return drop-down
            'direction': 'down',
            'showactive': True
        }
    ],
    legend=dict(
        orientation="h",  # Make legend horizontal
        yanchor="bottom",  # Anchor the legend to the top
        y=1.02,  # Position it above the plot
        xanchor="right",  # Align legend to the right
        x=1,  # Place the legend to the right of the title
    ),
    title="Impact of Expense Ratio and Return Rate on a $10,000 Investment",
    xaxis_title="Years",
    yaxis_title="Investment Value (USD)",
    template="plotly_dark"
)

# Show plot
fig.show()
